In [1]:
%matplotlib inline

In [2]:
import lsst.sims.skybrightness as sb
import lsst.sims.photUtils.Bandpass as Bandpass
import os
import numpy as np
import matplotlib.pylab as plt
import healpy as hp

In [3]:
sm = sb.SkyModel(mags=True, twilight=False, zodiacal=False, moon=True)
nside = 16
hpmap = np.zeros(hp.nside2npix(nside))
lat, ra = hp.pix2ang(nside, np.arange(hpmap.size))
dec = np.pi/2-lat
good = np.where(dec < 0)
dec = dec[good]
ra = ra[good]
print 'computing at %i points on the sky' % ra.size

computing at 1504 points on the sky


/Users/yoachim/gitRepos/sims_skybrightness/python/lsst/sims/skybrightness/interpComponents.py:76: RuntimeWarning: divide by zero encountered in log10
  self.logSpec = np.log10(self.spec['spectra'])


In [4]:
# Let's pull a large number of dark time mjds to test
data,ack = sb.allSkyDB(100, sqlQ='select mjd from dates where sunAlt < %f and moonAlt < %f' % (np.radians(-20.), 0.), 
                   dtypes=zip(['mjd'],[float]))

In [5]:
data['mjd']

array([ 56948.019861,  56948.020683,  56948.021493, ...,  57073.368067,
        57073.368877,  57073.369687])

In [6]:
import timeit
def runStuff(sm, mjd, n=10):
    for i in np.arange(n):
        sm.setRaDecMjd(ra,dec, mjd[i])
        sm.computeSpec()

npts = [10,100,1000]
timeResults = []
for npt in npts:
    time = timeit.timeit("runStuff(sm,data['mjd'], n=%i)" % npt, number=1, setup="from __main__ import runStuff,sm, data")
    timeResults.append(time)
    


/Users/yoachim/gitRepos/sims_skybrightness/python/lsst/sims/skybrightness/interpComponents.py:119: RuntimeWarning: divide by zero encountered in divide
  w1 = (self.dimDict['airmass'][right] - interpPoints['airmass'][order])/fullRange
/Users/yoachim/gitRepos/sims_skybrightness/python/lsst/sims/skybrightness/interpComponents.py:120: RuntimeWarning: divide by zero encountered in divide
  w2 = (interpPoints['airmass'][order] - self.dimDict['airmass'][left])/fullRange


In [7]:
timeResults

[0.03610801696777344, 0.3406941890716553, 3.401287794113159]

In [8]:
print 'Compute time for 2.5 million 6-filter, half-sky, moon down calcs = %f hours' % (timeResults[-1]/1e3*2.5e6/3600.)

Compute time for 2.5 million 6-filter, half-sky, moon down calcs = 2.362005 hours


In [9]:
# Now with the moon up
data,ack = sb.allSkyDB(100, sqlQ='select mjd from dates where sunAlt < %f and moonAlt > %f' % (np.radians(-20.), np.radians(10.)), 
                   dtypes=zip(['mjd'],[float]))

In [10]:
timeResults = []
for npt in npts:
    time = timeit.timeit("runStuff(sm,data['mjd'], n=%i)" % npt, number=1, setup="from __main__ import runStuff,sm, data")
    timeResults.append(time)
timeResults

/Users/yoachim/lsst/DarwinX86/healpy/1.8.1+6/lib/python/healpy-1.8.1-py2.7-macosx-10.5-x86_64.egg/healpy/pixelfunc.py:1045: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if phi == None:


[0.05707383155822754, 0.5562448501586914, 5.565908908843994]

In [11]:
print 'Compute time for 2.5 million 6-filter, half-sky, moon-up calcs = %f hours' % (timeResults[-1]/1e3*2.5e6/3600.)

Compute time for 2.5 million 6-filter, half-sky, moon-up calcs = 3.865215 hours
